<a href="https://colab.research.google.com/github/p31x070/GPT/blob/main/Copy_of_PDF_Query_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=TLf90ipMzfE

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "..."

In [4]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# location of the pdf file/files. 
reader = PdfReader('/content/drive/MyDrive/Zotero/Direito Penal/Fernández Vanegas&Fernández Vanegas_2021_Repensando el principio de legalidad penal.pdf')

In [32]:
reader

In [33]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

# raw_text

In [34]:
raw_text[:100]

'RevistadelaFacultad deDerecho, (50), ene-jun, 2021, e20215008\neISSN 2301-0665/DOI: 10.22187/rfd2021n'

In [35]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [36]:
len(texts)

93

In [37]:
texts[0]

'RevistadelaFacultad deDerecho, (50), ene-jun, 2021, e20215008\neISSN 2301-0665/DOI: 10.22187/rfd2021n50a8\nCreativeCommonsReconocimiento4.0Internacional LicenseAtribución CCBY\n1de33Resumen: La sociedad postindustrial generó la creación de nuevos riesgos que convi-\nven en nuestras actividades cotidianas y que pueden tener consecuencias fatales en nues-\ntra existencia. Entre las consecuencias producto de esta sociedad podemos observar la\ninevitable expansión del derecho penal, la creación desmedida de tipos penales de peligro\ny la flexibilización de garantías procesales. Estas circunstancias hacen tambalear, entre\notros, el principio de legalidad, que históricamente ha sido concebido como el cumpli-\nmiento estricto delos parámetros establecidos en la norma yuna limitación a la discrecio-\nnalidaddelos operadores dejusticia. Seha utilizado una metodología detipo exploratorio\nydescriptivo mediante el método histórico, el análisis lógico yel análisis exegético jurídi-'

In [38]:
texts[1]

'nalidaddelos operadores dejusticia. Seha utilizado una metodología detipo exploratorio\nydescriptivo mediante el método histórico, el análisis lógico yel análisis exegético jurídi-\nco, para determinarel modo en que la sociedadpostindustrial ha tenido una influencia di-\nrecta en la flexibilización de esta garantía, a tal punto que, en la práctica, está siendo\ncadavezmássacrificadaenprodecriteriosdeeficientísimopenalynoimpunidad.\nPalabras clave: Sociedad de riesgo, principio de legalidad, postindustrial, derecho\npenal, garantíasprocesales, justicia.\nAbstract:The post-industrial societygeneratedthe creation ofnewrisks thatcoexistin\nour daily activities and that can have fatal consequences in our existence. Among the\nconsequences resulting from this societywecan observetheinevitableexpansion ofcriminal\nlaw, the excessive creation of criminal types of danger and the relaxation of procedural\nProfesor de Derecho Procesal Penal de la Universidad de Guayaquil. ORCID:  0000-0001-5175-

In [39]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [40]:
docsearch = FAISS.from_texts(texts, embeddings)

In [41]:
docsearch

In [42]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [43]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [44]:
query = "Quem é o autor do artigo"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O autor do artigo é Héctor Gabriel Vanegas Fernández.'

In [49]:
query = "Crie um resumo do artigo"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Este artigo explora como a sociedade pós-industrial tem influenciado a flexibilização das garantias processuais, na medida em que, na prática, elas estão sendo cada vez mais sacrificadas em favor de critérios de eficiência criminal e não de impunidade. O artigo se concentra na luta contra a obscuridade da lei penal, onde normas muitas vezes abrangem situações abstratas de risco e nem sempre existe uma determinação volitiva de produzir dano ou uma probabilidade específica de direcionar a comissão de um resultado lesivo. O artigo também discute como muitas das condutas relevantes penais decorrentes da sociedade de risco estão sendo remetidas a normas administrativas ou a regulamentos privados.'

In [45]:
query = "Qual o tema do artigo?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O tema do artigo é o princípio da legalidade penal na sociedade de risco.'

In [47]:
query = "Quais as teses abordadas?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' As teses abordadas são relacionadas à erosão do princípio da legalidade em decorrência da sociedade de risco, a relativização do princípio da legalidade e as possíveis alternativas a ele apresentadas pelo direito penal do futuro.'

In [48]:
query = "Quais as conclusões a que chega o autor?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O autor conclui que a sociedade pós-industrial gerou a criação de novos riscos que coexistem nas nossas atividades diárias e que podem ter consequências fatais na nossa existência. Ele também conclui que a relativização ou má aplicação do princípio de legalidade não é uma evolução, pois na realidade é o contrário. O autor também conclui que a expectativa de justiça está erodindo o princípio de legalidade e, com o objetivo de que esses crimes não fiquem impunes, está sendo imposta uma redução das garantias.'

In [50]:
query = "como citar esse artigo?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Héctor Gabriel Vanegas Fernández. "Repensando el principio de legalidad penal: sociedad de riesgo y derecho penal", Revista de la Facultad de Derecho, (50), ene-jun, 2021, e20215008. DOI: 10.22187/rfd2021n50a8.'

In [51]:
query = "crie palavras chave para o artigo"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Sociedade de risco, princípio da legalidade, pós-industrial, direito penal, garantias processuais, justiça, reinterpretação, ampliação, flexibilização, relativização.'